In [36]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('data/direct_relations.csv')
data

,Definição_Synset,ID_Synset,Definição_Relacionada,ID_Relacionada,Relacao
0,an entity that has physical existence,physical_entity.n.01,that which is perceived or known or inferred t...,entity.n.01,Hypernyms
1,a general concept formed by extracting common ...,abstraction.n.06,that which is perceived or known or inferred t...,entity.n.01,Hypernyms
2,a separate and self-contained entity,thing.n.12,an entity that has physical existence,physical_entity.n.01,Hypernyms
3,a tangible and visible entity; an entity that ...,object.n.01,an entity that has physical existence,physical_entity.n.01,Hypernyms
4,an assemblage of parts that is regarded as a s...,whole.n.02,a tangible and visible entity; an entity that ...,object.n.01,Hypernyms
...,...,...,...,...,...
111271,become empty of water,run_dry.v.01,become dry or drier,dry.v.02,Hypernyms
111272,get foggy,fog_up.v.01,make overcast or cloudy,overcast.v.01,Hypernyms
111273,burn to charcoal,char.v.01,cause to burn or combust,burn.v.05,Hypernyms
111274,"become hazy, dull, or cloudy",haze.v.01,make overcast or cloudy,overcast.v.01,Hypernyms


In [3]:
data['Relacao'].value_counts()

Relacao
Hypernyms    89089
Holonyms     22187
Name: count, dtype: int64

In [49]:
entities = pd.concat([data['ID_Synset'], data['ID_Relacionada']]).unique()
print(entities.shape[0], entities)

mapping = {name : i for i, name in enumerate(entities)}
mapping

91294 ['physical_entity.n.01' 'abstraction.n.06' 'thing.n.12' ... 'burn.v.05'
 'absorb.v.06' 'blow.v.02']


{'physical_entity.n.01': 0,
 'abstraction.n.06': 1,
 'thing.n.12': 2,
 'object.n.01': 3,
 'whole.n.02': 4,
 'congener.n.03': 5,
 'living_thing.n.01': 6,
 'organism.n.01': 7,
 'benthos.n.02': 8,
 'dwarf.n.03': 9,
 'heterotroph.n.01': 10,
 'parent.n.02': 11,
 'life.n.10': 12,
 'biont.n.01': 13,
 'cell.n.02': 14,
 'causal_agent.n.01': 15,
 'person.n.01': 16,
 'animal.n.01': 17,
 'plant.n.02': 18,
 'native.n.03': 19,
 'natural_object.n.01': 20,
 'substance.n.01': 21,
 'substance.n.07': 22,
 'matter.n.03': 23,
 'food.n.01': 24,
 'nutrient.n.02': 25,
 'artifact.n.01': 26,
 'article.n.02': 27,
 'psychological_feature.n.01': 28,
 'cognition.n.01': 29,
 'motivation.n.01': 30,
 'attribute.n.02': 31,
 'state.n.02': 32,
 'feeling.n.01': 33,
 'location.n.01': 34,
 'shape.n.02': 35,
 'time.n.05': 36,
 'space.n.01': 37,
 'absolute_space.n.01': 38,
 'phase_space.n.01': 39,
 'event.n.01': 40,
 'process.n.06': 41,
 'act.n.02': 42,
 'group.n.01': 43,
 'relation.n.01': 44,
 'possession.n.02': 45,
 'social

## PyG's Wordnet 

In [38]:
import torch 
from torch_geometric.datasets import WordNet18RR

wordnet = WordNet18RR(root='data/wordnet18rr', force_reload=True)

Processing...
Done!


In [39]:
edge_index = wordnet.edge_index
edge_type  = wordnet.edge_type 
train_mask, val_mask, test_mask = wordnet.train_mask, wordnet.val_mask, wordnet.test_mask

In [47]:
edge_type.shape, edge_type

(torch.Size([93003]), tensor([ 3,  9, 10,  ...,  3,  3,  3]))

In [40]:
wordnet.edge2id

{'_also_see': 0,
 '_derivationally_related_form': 1,
 '_has_part': 2,
 '_hypernym': 3,
 '_instance_hypernym': 4,
 '_member_meronym': 5,
 '_member_of_domain_region': 6,
 '_member_of_domain_usage': 7,
 '_similar_to': 8,
 '_synset_domain_topic_of': 9,
 '_verb_group': 10}